# Model dimensionality
PyBaMM currently supports three types of model dimensions for the current collectors: 0D, 1D and 2D. These can be configured using the "dimensionality" model option.


If this notebook is running on Google Colab, we first need to install PyBaMM (you can skip if not using Google Colab).

In [1]:
%pip install pybamm -q

You should consider upgrading via the '/home/ferranbrosa/continuum-model-review/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


We start importing the relevant packages

In [2]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

We now define the model of interest, in this case we set a 0D model for the current collectors

In [3]:
model_0D = pybamm.lithium_ion.SPMe({"dimensionality": 0})

Then we import the parameters, and we modify the values of all heat transfer coefficients

In [4]:
param = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Marquis2019)
for p in param.keys():
    if 'heat transfer coefficient' in p:
        param.update({p:1.0})

We can check that the parameter values were indeed updated

In [5]:
param.search('heat transfer coefficient')

Edge heat transfer coefficient [W.m-2.K-1]	1.0
Negative current collector surface heat transfer coefficient [W.m-2.K-1]	1.0
Negative tab heat transfer coefficient [W.m-2.K-1]	1.0
Positive current collector surface heat transfer coefficient [W.m-2.K-1]	1.0
Positive tab heat transfer coefficient [W.m-2.K-1]	1.0
Total heat transfer coefficient [W.m-2.K-1]	1.0


Finally we can solve and plot the simulation

In [6]:
sim_0D = pybamm.Simulation(model_0D, parameter_values=param)
sim_0D.solve([0, 3600])

In [7]:
sim_0D.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

The default dimensionality for the current collectors is 0 but has been specified above for demonstration purposes. With this option the Ohmic losses in the current collectors are assumed to be negligible meaning that the entire current collector is held at the same potential. This may be a good approximation for smaller pouch cells or even larger ones at low C-rates. However, for cells with larger aspect ratios the Ohmic losses in the current collectors become more important and regions of the cell may discharge or charge at different rates depending on their proximity to a tab. 

We can repeat the simulation but now with a 1D model for the current collectors.

In [8]:
model_1D = pybamm.lithium_ion.SPMe({"dimensionality": 1,
                                    "current collector": "potential pair"})
sim_1D = pybamm.Simulation(model_1D, parameter_values=param)
sim_1D.solve([0, 3600])
sim_1D.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Now we can observe the effects of dimensionality on a thermal model. In this case, we choose a lumped thermal model.

In [9]:
model_1D_lumped = pybamm.lithium_ion.SPMe({"dimensionality": 1,
                                           "current collector": "potential pair",
                                           "thermal": "lumped"})
sim_1D_lumped = pybamm.Simulation(model_1D_lumped, parameter_values=param)
sim_1D_lumped.solve([0, 3600])

In [10]:
temp = sim_1D_lumped.solution["Cell temperature [K]"].entries
t_min = temp.min()
t_max = temp.max()
variable_limits = {"X-averaged cell temperature [K]": (t_min, t_max),
                   "Cell temperature [K]": (t_min, t_max),
                   "Volume-averaged cell temperature [K]": (t_min, t_max)}

sim_1D_lumped.plot([
    #"Terminal voltage [V]",
                    "X-averaged cell temperature [K]",
                    "Cell temperature [K]",
                    "Volume-averaged cell temperature [K]"], variable_limits=variable_limits)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

And we can proceed similarly but with an `x-lumped` thermal model instead (more details can be found in the [PyBaMM documentation](https://pybamm.readthedocs.io/en/latest/source/models/submodels/thermal/index.html))

In [11]:
model_1D_x_lumped = pybamm.lithium_ion.SPMe({"dimensionality": 1,
                                             "current collector": "potential pair",
                                             "thermal": "x-lumped"})
sim_1D_x_lumped = pybamm.Simulation(model_1D_x_lumped, parameter_values=param)
sim_1D_x_lumped.solve([0, 3600])

In [12]:
temp = sim_1D_x_lumped.solution["Cell temperature [K]"].entries
t_min = temp.min()
t_max = temp.max()
variable_limits = {"X-averaged cell temperature [K]": (t_min, t_max),
                   "Cell temperature [K]": (t_min, t_max),
                   "Volume-averaged cell temperature [K]": (t_min, t_max)}
sim_1D_x_lumped.plot(["X-averaged cell temperature [K]",
                      "Cell temperature [K]",
                      "Volume-averaged cell temperature [K]"],
                     #variable_limits=variable_limits,
                     variable_limits='tight',
                    )

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Finally we can solve the model with a 2D current collector model, which captures the effects of the current collectors.

In [13]:
model_2D = pybamm.lithium_ion.SPMe({"dimensionality": 2,
                                    "current collector": "potential pair"})
sim_2D = pybamm.Simulation(model_2D, parameter_values=param)
sim_2D.solve([0, 3600])

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [14]:
sim_2D.plot(["Negative current collector potential [V]",
             "Positive current collector potential [V]",
             "Current collector current density [A.m-2]",
             "Local voltage [V]"], variable_limits="tight")

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

The articles that contributed to this notebook are

In [15]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Tom Gustafsson and Geordie D. McBain. scikit-fem: A Python package for finite element assembly. Journal of Open Source Software, 5(52):2369, 2020. doi:10.21105/joss.02369.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/2.0341915jes.
[5] Valentin Sulzer, Scott G. Marq